In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
! pip install hazm

In [ ]:
import collections
from hazm import *
from matplotlib import pyplot as plt
import json
#from parsivar import *
import operator
import numpy as np

In [ ]:
punctuation = [':', '،', '.', ')', '(', '}', '{', '؟', '!', '-', '/', '؛', '#', '*', '\n', '\"',
                ']', '[', '«', '»', '٪', '+', '٠', '\\', '\"', '_', '\'']


sign = ['%', '@', '_', "\"", '$', '&', ',', '"', '>', '<', '|',
                         '­', ';', 'é', '=', '+', '?']

numbers = ['0', '1', '2', '3', '4',
                         '5', '6', '7', '8', '9', '۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹']

english_chars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
                   'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
                   'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

escape_chars = ['\u200c', '\u200b', '\u200f']

clear_list = [sign, numbers, english_chars, escape_chars]

In [ ]:
def split_word(word):
  return [c for c in word]

def clear_content(content_to_clear):
    
    for p in punctuation:
        content_to_clear = content_to_clear.replace(p, " ")
    
    for i in clear_list:

      for n in i:
        content_to_clear = content_to_clear.replace(n, '')

    return content_to_clear

In [ ]:


with open("/content/gdrive/MyDrive/IR_data_news_12k.json", 'r', encoding='utf-8') as readfile:
  data = json.load(readfile)
    
positional_index = {}
doc_id_title = {}
term_frq_per_doc = []
term_frq = {}
doc_id_url = {}



stop_words = stopwords_list()
stemmer = Stemmer()
lemmatizer = Lemmatizer()


In [ ]:
def Zipfs_law(term_frq):
    print("hi")
    term_frq = dict(sorted(term_frq.items(), key=lambda item: item[1], reverse=True))
    log_rank = np.log10(np.linspace(1, len(term_frq), len(term_frq)))
    log_cf = np.log10(np.array(list(term_frq.values())))
    plt.plot(log_rank, log_cf, 'b')
    term_frq_values = list(term_frq.values())
    cf1 = term_frq_values[0]
    log_i = np.log10(np.linspace(2, len(term_frq), len(term_frq) - 1))
    log_cfi = np.log10(cf1) - log_i
    plt.plot(log_i, log_cfi, 'r')

    #plt.title("Checking Zipf's Law after removing stop words")
    plt.title("Checking Zipf's Law before removing stop words")
    plt.legend(["Actual Values", "Zipf's Law"])
    plt.xlabel("log10 rank")
    plt.ylabel("log10 cf")
    plt.show()

In [ ]:
def heaps_law(p1, p2, p3, p4, p5, p6, p7, p8, p9, p10):
    log_M = np.log10([p2, p4, p6, p8])
    log_T = np.log10([p1, p3, p5, p7])

    main_M = p10
    main_T = p9

    plt.plot(log_T, log_M, 'b')
    [b, log_k] = np.polyfit(log_T, log_M, 1)
    plt.plot(log_T, b * log_T + log_k, 'r')

    plt.title("Checking Heaps' Law after finding roots")
    plt.legend(["Actual Values after finding roots", "Heaps' Law after finding roots"])
    plt.xlabel("log10 T")
    plt.ylabel("log10 M")
    plt.show()

    log_main_T = np.log10(main_T)
    predict_log_main_M = log_k + b * log_main_T
    predict_main_M = np.power(10, predict_log_main_M)

    k = np.power(10, log_k)

    print("k = ", k)
    print("b = ", b)
    print("Number of Tokens: ", main_T)
    print("Predicted Size of Vocabulary : ", predict_main_M)
    print("Actual Size of Dictionary : ", main_M)


In [ ]:
count = 0
doc_id = -1
for row in data:
    dictionary = dict()
    term_position = dict()
    count = count + 1

    doc_id = doc_id + 1

    doc_id_title[doc_id] = data[row]['title']
    doc_id_url[doc_id] = data[row]['url']


    content = data[row]['content']

    content = clear_content(content)

    terms = word_tokenize(content)
    counter = 0



    # removing stop words
    for words in stop_words:
        for term in terms:
            if term == words:
                counter = counter + 1
        for i in range(counter):
            terms.remove(words)
        counter = 0

    final_terms = list()

    for term in terms:
    #res = stemmer.stem(term)
    #res = lemmatizer.lemmatize(res)
    #final_terms.append(res)
    # used while ignoring stemming for heaps law calculations
      final_terms.append(term)

    for i in final_terms:
        position_list = list()
        if i in dictionary:
            pass
        else:
            position_counter = 0
            for j in final_terms:
                position_counter = position_counter + 1
                if i == j:
                    frq = dictionary.get(i, 0)
                    dictionary[i] = frq + 1
                    position_list = term_position.get(i, [])
                    position_list.append(position_counter)
                    term_position[i] = position_list

    term_frq_per_doc.append(dictionary)

    # creating positional index
    for term in term_position.keys():
        if term in positional_index.keys():
            dic1 = positional_index.get(term, {})
            dic1[doc_id] = term_position[term]
            positional_index[term] = dic1
            # pass
        else:
            doc_list = dict()
            doc_list[doc_id] = term_position[term]
            positional_index[term] = doc_list

positional_index = collections.OrderedDict(sorted(positional_index.items()))

In [ ]:
frequency = 0
for term in positional_index.keys():
    for words_per_doc in term_frq_per_doc:
        if term in words_per_doc:
            frequency = frequency + words_per_doc[term]
    term_frq[term] = frequency
    frequency = 0

the_first_500_tokens = list()
the_first_1000_tokens = list()
the_first_1500_tokens = list()
the_first_2000_tokens = list()
for term in positional_index:
    for docid in positional_index[term]:
        if docid < 500:
            the_first_500_tokens.append(term)

for term in positional_index:
    for docid in positional_index[term]:
        if docid < 1000:
            the_first_1000_tokens.append(term)

for term in positional_index:
    for docid in positional_index[term]:
        if docid < 1500:
            the_first_1500_tokens.append(term)

for term in positional_index:
    for docid in positional_index[term]:
        if docid < 2000:
            the_first_2000_tokens.append(term)


the_first_500_terms = set(the_first_500_tokens)
the_first_1000_terms = set(the_first_1000_tokens)
the_first_1500_terms = set(the_first_1500_tokens)
the_first_2000_terms = set(the_first_2000_tokens)

all_tokens = positional_index
all_tokens_len = 0
for term in positional_index:
    all_tokens_len += len(positional_index[term].values())

all_terms = len(positional_index.keys())

p1 = len(the_first_500_tokens)
p2 = len(the_first_500_terms)
p3 = len(the_first_1000_tokens)
p4 = len(the_first_1000_terms)
p5 = len(the_first_1500_tokens)
p6 = len(the_first_1500_terms)
p7 = len(the_first_2000_tokens)
p8 = len(the_first_2000_terms)
p9 = all_tokens_len
p10 = all_terms


print("Tokens of the first 500 docs ")
print(p1)
print("terms of the first 500 docs ")
print(p2)
print("Tokens of the first 1000 docs ")
print(p3)
print("terms of the first 1000 docs ")
print(p4)
print("Tokens of the first 1500 docs ")
print(p5)
print("terms of the first 1500 docs ")
print(p6)
print("Tokens of the first 2000 docs ")
print(p7)
print("terms of the first 2000 docs ")
print(p8)
print("Tokens of all the docs")
print(p9)
print("terms of all the docs")
print(p10)

Zipfs_law(term_frq)

heaps_law(p1, p2, p3, p4, p5, p6, p7, p8, p9, p10)


In [ ]:
with open("/content/gdrive/MyDrive/positional_index.json", "w") as write_file:
   json.dump(positional_index, write_file)

with open("/content/gdrive/MyDrive/doc_id_title.json", "w") as write_file:
   json.dump(doc_id_title, write_file)

with open("/content/gdrive/MyDrive/doc_id_url.json", "w") as write_file:
   json.dump(doc_id_url, write_file)

with open("/content/gdrive/MyDrive/term_frq.json", "w") as write_file:
   json.dump(term_frq, write_file)

with open("/content/gdrive/MyDrive/term_frq_per_doc.json", "w") as write_file:
   json.dump(term_frq_per_doc, write_file)

In [ ]:
stop_words = stopwords_list()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

In [ ]:
with open("/content/gdrive/MyDrive/positional_index.json", "r") as read_file:
    positional_index = json.load(read_file)

with open("/content/gdrive/MyDrive/doc_id_title.json", "r") as read_file:
    doc_id_title = json.load(read_file)

with open("/content/gdrive/MyDrive/doc_id_url.json", "r") as read_file:
    doc_id_url = json.load(read_file)

In [ ]:
query = "دانشگاه ~امیرکبیر"

query = clear_content(query)

query_list = query.split()
new_list = []
remove_words = []

for i in query_list:
  if i[0] == "~":
    remove_words.append(i.replace('~', ''))
    continue
  new_list.append(i)

query_list = new_list

q_l = list(query_list)
query_list.clear()
for term in q_l:
    res = stemmer.stem(term)
    res = lemmatizer.lemmatize(res)
    query_list.append(res)

In [ ]:
# One Word Query
if len(query_list) == 1:
    docs_to_remove = list()
    for term in remove_words:
        docs_to_remove = list(positional_index[term].keys())
    if query_list[0] in positional_index.keys():
        for i in positional_index[query_list[0]]:
            if i not in docs_to_remove:
                print(doc_id_title[i])
                print(doc_id_url[i])

In [ ]:
states = []
for i in range(0, 2**len(query_list)):
  new_list = query_list.copy()
  dictionary = dict()
  binary_format = split_word(format(i, "b"))
  while len(binary_format) != len(query_list):
    binary_format.insert(0, '0')

  for j in range(0, len(binary_format)):
    if binary_format[j] == '1':
      new_list[j] = '0'
  zero_count = 0
  for n in new_list:
    if n == '0':
      zero_count += 1
  for i in range(zero_count):
    new_list.remove('0')
  dictionary[len(new_list)] = new_list
  states.append(dictionary)

In [ ]:
count = len(query_list)
counter = 0
docs_to_retrieve = []
title_results = []
while count != 1:
    for dict1 in states:
        k = list(dict1.keys())
        if k[0] == count:
            # Start
            positional_postings_lists = list()
            doc_id_list = list()
            position_list = list()
            dict1_values = list(dict1.values())
            word_list = list(dict1_values[0])
            for term in word_list:
                id_position_per_term = dict()
                for row in positional_index:
                    if row == term:
                        doc_ids_list = list(positional_index[row].keys())
                        for word_doc in doc_ids_list:
                            id_position_per_term[word_doc] = list(positional_index[row][word_doc])
                positional_postings_lists.append(id_position_per_term)

            # Start Algorithm
            end = 0
            doc_id = 0
            flag = 0
            flag3 = 0
            check = 1
            keys = list(positional_postings_lists[0].keys())
            if len(keys) != 0:
                last_doc_id_term_1 = keys[len(keys) - 1]
                last_doc_id_term_1 = int(last_doc_id_term_1)
            else:
                end = 1

            while True:
                if end == 1:
                    end = 0
                    break
                if doc_id > last_doc_id_term_1:
                    break
                d_i = 0
                for d_i in positional_postings_lists[0]:
                    d_i = int(d_i)
                    if doc_id > d_i:
                        continue
                    for term in positional_postings_lists:
                        if term == positional_postings_lists[0]:
                            continue
                        d_i = str(d_i)
                        if d_i in term.keys():
                            pass
                        else:
                            flag = -1
                    if flag == -1:
                        flag = 0
                        check = check + 1
                        continue
                    else:
                        flag3 = 1
                        d_i = int(d_i)
                        doc_id = d_i
                        break

                d_i = int(d_i)

                if check == (len(positional_postings_lists[0]) + 1):
                    check = 1
                    break

                if flag3 == 0 and doc_id == last_doc_id_term_1:
                    break

                if flag3 == 0 and d_i == last_doc_id_term_1:
                    break
                flag3 = 0

                d_i = str(d_i)

                doc_id = str(doc_id)
                if doc_id not in positional_postings_lists[0].keys():
                    break
                term_1_positions_per_doc_id = positional_postings_lists[0][doc_id]
                flag1 = 0
                flag2 = 0
                list_found_docs = list()
                for o in term_1_positions_per_doc_id:
                    o = int(o)
                    pos = o
                    for term in positional_postings_lists:
                        if term == positional_postings_lists[0]:
                            continue
                        pos = pos + 1
                        v = term[doc_id]
                        pos = str(pos)
                        if pos in v:
                            pass
                        else:
                            flag1 = -1
                        pos = int(pos)
                    if flag == -1:
                        flag = 0
                        continue
                    else:
                        list_found_docs.append(doc_id)
                        docs_to_retrieve.append(doc_id)
                        break

                doc_id = int(doc_id)
                doc_id = doc_id + 1

            # End Algorithm
            # End
    count = count - 1

In [ ]:
for term in remove_words:
    docs_to_remove = list(positional_index[term].keys())
    for d in docs_to_remove:
        if d in docs_to_retrieve:
            docs_to_retrieve.remove(d)

for i in docs_to_retrieve:
    print(doc_id_title[i])
    print(doc_id_url[i])


In [ ]:
def tfidf(tf, df, N):
    if tf * df == 0:
        return 0.
    return (1 + np.log10(tf)) * np.log10(1. * N / df)


with open("/content/gdrive/MyDrive/positional_index.json", "r") as read_file:
    positional_index = json.load(read_file)

with open("/content/gdrive/MyDrive/doc_id_title.json", "r") as read_file:
    doc_id_title = json.load(read_file)

with open("/content/gdrive/MyDrive/term_frq_per_doc.json", "r") as read_file:
    term_frq_per_doc = json.load(read_file)

term_df = dict()
for term in positional_index:
    term_df[term] = len(positional_index[term].keys())

N = len(doc_id_title.keys())
lengths = np.zeros(N)
for term in positional_index:
    df = term_df[term]
    docs = positional_index[term].keys()
    for doc_id in docs:
        doc_id = int(doc_id)
        if term not in term_frq_per_doc[doc_id]:
            continue
        tf = term_frq_per_doc[doc_id][term]
        weight = tfidf(tf, df, N)
        lengths[doc_id] += weight ** 2
lengths = np.sqrt(lengths)
lengths = list(lengths)
with open("/content/gdrive/MyDrive/lengths.json", "w") as write_file:
    json.dump(lengths, write_file)

In [ ]:
with open("/content/gdrive/MyDrive/doc_id_title.json", "r") as read_file:
    doc_id_title = json.load(read_file)

with open("/content/gdrive/MyDrive/positional_index.json", "r") as read_file:
    positional_index = json.load(read_file)

with open("/content/gdrive/MyDrive/term_frq_per_doc.json", "r") as read_file:
    term_frq_per_doc = json.load(read_file)

r = input("Enter r: ")
r = int(r)
postings_lists = dict()

for term in positional_index:
    doc_id_tf = dict()
    for doc_id in range(0, len(doc_id_title)):
        if term in term_frq_per_doc[doc_id]:
            doc_id_tf[doc_id] = term_frq_per_doc[doc_id][term]

    res = sorted(doc_id_tf.items(), key=operator.itemgetter(1), reverse=True)[:r]
    postings_lists[term] = [t[0] for t in res]

with open("/content/gdrive/MyDrive/postings_lists.json", "w") as write_file:
    json.dump(postings_lists, write_file)

In [ ]:
with open("/content/gdrive/MyDrive/positional_index.json", "r") as read_file:
    positional_index = json.load(read_file)

with open("/content/gdrive/MyDrive/doc_id_title.json", "r") as read_file:
    doc_id_title = json.load(read_file)

with open("/content/gdrive/MyDrive/term_frq_per_doc.json", "r") as read_file:
    term_frq_per_doc = json.load(read_file)

with open("/content/gdrive/MyDrive/doc_id_url.json", "r") as read_file:
    doc_id_url = json.load(read_file)

In [ ]:
def split(word):
    return [char for char in word]


def query_parser(query):

    query = clear_content(query)

    query_list = query.split()


    for words in stop_words:
        for term in query_list:
            if term == words:
                query_list.remove(words)

    q_l = list(query_list)
    query_list.clear()
    for term in q_l:
        res = stemmer.stem(term)
        res = lemmatizer.lemmatize(res)
        query_list.append(res)
    return query_list


def tfidf(tf, df, N):
    if tf * df == 0:
        return 0.
    return (1 + np.log10(tf)) * np.log10(1. * N / df)

def champion_lists_create(status, postings_list):
    if status == False:
        for term in positional_index:
            postings_list[term] = list(positional_index[term].keys())
        return postings_list
    else:
        with open("/content/gdrive/MyDrive/postings_lists.json", "r") as read_file:
            postings_list = json.load(read_file)
        return postings_list

In [ ]:
term_df = dict()
for term in positional_index:
    term_df[term] = len(positional_index[term].keys())

postings_list = dict()
# showing top 10 results
k = 15

# if champion_lists_status is set True we use Champion lists if is set to False we use the default Postings lists
champion_lists_status = True
postings_list = champion_lists_create(champion_lists_status, postings_list)
query = "لیگ فوتبال آسیا"
query = query_parser(query)
N = len(doc_id_title.keys())

with open("/content/gdrive/MyDrive/lengths.json", "r") as read_file:
    lengths = json.load(read_file)

scores = dict()
for qt in query:
    df = term_df[qt]
    qt_tf = query.count(qt)
    qt_weight = tfidf(df, qt_tf, N)
    qt_postings = postings_list[qt]
    for doc_id in qt_postings:
        doc_id = int(doc_id)
        if lengths[doc_id] == 0:
            continue
        doc_tf = term_frq_per_doc[doc_id][qt]
        doc_weight = tfidf(df, doc_tf, N)
        try:
            scores[doc_id] += qt_weight * doc_weight
        except KeyError:
            scores[doc_id] = qt_weight * doc_weight


for doc_id in scores:
    scores[doc_id] /= lengths[doc_id]

scores = dict(sorted(scores.items(), key=operator.itemgetter(1), reverse=True))
docs_to_retrieve = list(scores.keys())[:k]
for i in docs_to_retrieve:
    i = str(i)
    print(f"{doc_id_url[i]}\n"
          f"doc id: \n{i}\n"
          f"Title: \n{doc_id_title[i]}")